In [31]:
import requests
import json
import urllib

base_url= "https://maps.googleapis.com/maps/api/geocode/json?"
AUTH_KEY = ""

# set up your search parameters - address and API key
parameters = {"address": "Venice Blvd. and Moore st, Los Angeles",
              "key": AUTH_KEY}

print(f"{base_url}{urllib.parse.urlencode(parameters)}")

r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")
data = json.loads(r.content)

In [30]:
data

{'results': [{'address_components': [{'long_name': 'Venice Boulevard & Moore Street',
     'short_name': 'Venice Blvd. & Moore St',
     'types': ['intersection']},
    {'long_name': 'Mar Vista',
     'short_name': 'Mar Vista',
     'types': ['neighborhood', 'political']},
    {'long_name': 'Los Angeles',
     'short_name': 'Los Angeles',
     'types': ['locality', 'political']},
    {'long_name': 'Los Angeles County',
     'short_name': 'Los Angeles County',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'California',
     'short_name': 'CA',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'United States',
     'short_name': 'US',
     'types': ['country', 'political']},
    {'long_name': '90066', 'short_name': '90066', 'types': ['postal_code']}],
   'formatted_address': 'Venice Blvd. & Moore St, Los Angeles, CA 90066, USA',
   'geometry': {'location': {'lat': 34.0001831, 'lng': -118.4404026},
    'location_type': 'GEOMETR

# Latitude & Longitude


In [6]:
lat = data['results'][0]['geometry']['location']['lat']
lng = data['results'][0]['geometry']['location']['lng']
print( str(lat) + "," + str(lng))

34.0001831,-118.4404026


# Street and Cross Street 

In [7]:
data['results'][0]['formatted_address']

'Venice Blvd. & Moore St, Los Angeles, CA 90066, USA'

## Check if the cross streets exist with GEOMETRIC_CENTER or APPROXIMATE

In [11]:
data['results'][0]['geometry']['location_type'] 

'GEOMETRIC_CENTER'

In [13]:
def geo_street(street_01, street_02):
    parameters = {"address": street_01+" and " + street_02 +", Los Angeles",
              "key": AUTH_KEY}

    r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")
    
    data = json.loads(r.content)
    if data['status'] == 'OK' and  data['results'][0]['geometry']['location_type'] == 'GEOMETRIC_CENTER':
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        formatted_address = data['results'][0]['address_components'][0]['long_name']
        geo_type = data['results'][0]['geometry']['location_type']
        zipcode = data['results'][0]['address_components'][6]['long_name']
        area = data['results'][0]['address_components'][1]['long_name']
    elif  data['status'] == 'OK' and  data['results'][0]['geometry']['location_type'] == 'APPROXIMATE':
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        formatted_address = data['results'][0]['address_components'][0]['long_name']
        geo_type = data['results'][0]['geometry']['location_type']
        zipcode = np.nan
        area = data['results'][0]['address_components'][1]['long_name']
    elif data['status'] == 'ZERO_RESULTS':
        lat = np.nan
        lng = np.nan
        formatted_address = np.nan
        geo_type = np.nan
        zipcode = np.nan
        area = np.nan
        
    return lat, lng,formatted_address, zipcode, geo_type, area

In [15]:
v,w,x,y,z,zz = geo_street('sunset', 'vine')
print(v,w,x,y,z,zz)

34.0980023 -118.3266816 Sunset Boulevard & Vine Street 90028 GEOMETRIC_CENTER Central LA


In [25]:
import pandas as pd
import numpy as np
df = pd.read_csv ('../ykimdeveloper_LA_streets.csv')
# columns needs to be rename

In [26]:
#get ordinal street name, 41 becomes 41st
def ordinaltg(n):
    return str(n) + {1: 'st', 2: 'nd', 3: 'rd'}.get(4 if 10 <= n % 100 < 20 else n % 10, "th")

In [27]:

location_between = []
for col in df['Location Between']:        
    if type(col) is str and col.isnumeric() :
        int_col = int(col)
        location_between.append(ordinaltg(int_col))
    elif type(col) is str:
        col = col.replace(".", "").lower().rstrip()
        col = col.replace("_", " ")
        location_between.append(col)
    else:
        location_between.append(col)



# These streets needed to be manually changed because of human error 

In [28]:
df['Location Between'] = location_between
df['Location Between'][100] = 'moore st'
df['Location Between'][395] = "north fickett street"

/tmp/ipykernel_10468/414245053.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location Between'][100] = 'moore st'
/tmp/ipykernel_10468/414245053.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location Between'][395] = "north fickett street"


In [29]:
lat_arr = []
lng_arr = []
frmt_address_arr = []
zipcode_arr = []
area_arr = []
geo_type = []
for a,b in zip(df.Location, df['Location Between']): 
    if isinstance(a, str) and isinstance(b, str):
        lat, lng, frmt_address, zipcode, geotype, area = geo_street(a, b)
        print(lat, lng, frmt_address, zipcode, geotype, area)
        lat_arr.append(lat)
        lng_arr.append(lng)
        frmt_address_arr.append(frmt_address)
        zipcode_arr.append(zipcode)
        geo_type.append(geotype)
        area_arr.append(area)
    else:
        lat_arr.append(np.nan)
        lng_arr.append(np.nan)
        frmt_address_arr.append(np.nan)
        zipcode_arr.append(np.nan)
        area_arr.append(np.nan)
        geo_type.append(np.nan)

34.2587903 -118.4321397 Laurel Canyon Boulevard & Van Nuys Boulevard 91331 GEOMETRIC_CENTER Pacoima
34.2587903 -118.4321397 Laurel Canyon Boulevard & Van Nuys Boulevard 91331 GEOMETRIC_CENTER Pacoima
34.2587903 -118.4321397 Laurel Canyon Boulevard & Van Nuys Boulevard 91331 GEOMETRIC_CENTER Pacoima
34.2660955 -118.4221176 San Fernando Road & Van Nuys Boulevard 91331 GEOMETRIC_CENTER Pacoima
34.2587903 -118.4321397 Laurel Canyon Boulevard & Van Nuys Boulevard 91331 GEOMETRIC_CENTER Pacoima
34.2587903 -118.4321397 Laurel Canyon Boulevard & Van Nuys Boulevard 91331 GEOMETRIC_CENTER Pacoima
34.2660955 -118.4221176 San Fernando Road & Van Nuys Boulevard 91331 GEOMETRIC_CENTER Pacoima
34.2587903 -118.4321397 Laurel Canyon Boulevard & Van Nuys Boulevard 91331 GEOMETRIC_CENTER Pacoima
34.2587903 -118.4321397 Laurel Canyon Boulevard & Van Nuys Boulevard 91331 GEOMETRIC_CENTER Pacoima
34.2587903 -118.4321397 Laurel Canyon Boulevard & Van Nuys Boulevard 91331 GEOMETRIC_CENTER Pacoima
34.2587903 -

KeyboardInterrupt: 